In [5]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
def get_files_from_directory(directory, ext='.lua'):
    """
    遍历指定的目录，并返回所有.lua文件的完整路径。
    """
    target_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(ext):
                target_files.append(os.path.join(root, file))
    return target_files

lua_files = get_files_from_directory('/home/uework/AiWeb/pwz/github/data/lua', '.lua')\

df = pd.DataFrame(lua_files, columns=['file'])
content = []
for file in tqdm(lua_files):
    with open(file, 'r') as f:
        content.append(f.read())
df['content'] = content
df.head()

df.to_csv('/home/uework/AiWeb/pwz/github/data/lua/data.csv', index=False)

  0%|          | 0/397 [00:00<?, ?it/s]

100%|██████████| 397/397 [00:00<00:00, 8002.70it/s]


In [5]:
from datasets import load_from_disk, load_dataset

dataset = load_dataset("csv", data_files='/home/uework/AiWeb/pwz/github/data/dataset/dialogue-lua/data_cn.csv')
# add 'completion' as the features
dataset = dataset.map(lambda x: {'completion': x['response']})
dataset = dataset['train'].train_test_split(test_size=0.1)
dataset.save_to_disk('/home/uework/AiWeb/pwz/github/data/dataset/dialogue-lua')

Found cached dataset csv (/home/uework/.cache/huggingface/datasets/csv/default-17402498c2acedab/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 1/1 [00:00<00:00, 360.65it/s]
Loading cached processed dataset at /home/uework/.cache/huggingface/datasets/csv/default-17402498c2acedab/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-51e9ca12ce3c832c.arrow


In [ ]:
#################################################################
#### load large dataset using dask df
import dask.dataframe as dd

# File paths
file_paths = '/home/uework/AiWeb/pwz/github/data/lua/starcoder-data/train-*.parquet'

# Load data
ddf = dd.read_parquet(file_paths)

display(ddf.head())

import pandas as pd
ddf_code = dd.from_pandas(pd.read_csv('/home/uework/AiWeb/pwz/github/data/lua/shengqu/data.csv'),npartitions=10)
ddf_code.head()

ddf = dd.concat([ddf['content'], ddf_code['content']])

import dask.dataframe as dd
from datasets import Dataset
# Compute Dask DataFrame to get pandas DataFrame
pdf = ddf.compute()

# Convert pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(pdf)

dataset.save_to_disk('/home/uework/AiWeb/pwz/github/data/dataset/all-lua')


In [15]:
#################################################################
#### generate lua files from prompt and completion
from utils import dialogue_to_lua
# create dialogue
dialogue_df = pd.read_csv('/home/uework/AiWeb/pwz/github/data/dataset/dialogue-lua/data_cn.csv')
dialogue_to_lua(dialogue_df)

In [3]:
from datasets import load_from_disk

dataset = load_from_disk("/home/uework/AiWeb/pwz/github/data/dataset/openassistant")


In [8]:
import pandas as pd
train_df = dataset['train'].to_pandas()
display(train_df.head()['text'][0])


'### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.### Assistant: "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.\n\nRecent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining power, lead

In [ ]:
from utils import *
import pandas as pd
from tqdm import tqdm
from datasets import load_from_disk
from datasets import DatasetDict,Dataset
from scripts.preprocessing import clean_dataset
#################################################################
#### generate dataset for luacoder

dataset_folder = '/home/uework/AiWeb/pwz/github/data/dataset'
dataset_name = 'test-lua'
model_path = '/home/uework/AiWeb/pwz/github/model/starcoderbase-3b'
files_directory = ['/home/uework/AiWeb/pwz/github/data/lua/api-doc-lua',
                   '/home/uework/AiWeb/pwz/github/data/lua/shengqu',
                   '/home/uework/AiWeb/pwz/github/data/lua/dialogue'
                   ]
generate_luacoder_dataset(dataset_folder,dataset_name,model_path,files_directory)

In [12]:
import re

lua_code = '''
function PAR.OnShopOpen()
    local activeProfile = PA_SavedVars.General.activeProfile

    -- check if addon is enabled
	if PA_SavedVars.Repair[activeProfile].enabled == true then
		-- early check if there is something to repair
		if GetRepairAllCost() > 0 then
			-- check if equipped items shall be repaired
			if PA_SavedVars.Repair[activeProfile].equipped then
				PAR.RepairItems(BAG_WORN, PA_SavedVars.Repair[activeProfile].equippedThreshold)
			end
			-- check if backpack items shall be repaired
			if PA_SavedVars.Repair[activeProfile].backpack then
				PAR.RepairItems(BAG_BACKPACK, PA_SavedVars.Repair[activeProfile].backpackThreshold)
			end
		else
			if (not PA_SavedVars.Repair[activeProfile].hideNoRepairMsg) then
				PAR.println("PAR_NoRepair")
			end
		end
	end
end
'''

matches = re.findall(r'(--[^\n]*)\n\s*([^\n]+)', lua_code)

for match in matches:
    notation, line_code = match
    # Exclude lines of code that are comments
    if not line_code.strip().startswith('--'):
        start_idx = lua_code.index(line_code)
        end_idx = start_idx + len(line_code) - 1
        print(f"notation: {notation.strip()}")
        print(f"line code: {line_code.strip()}")
        print(f"start index: {start_idx}, end index: {end_idx}")
        print("----")


notation: -- check if addon is enabled
line code: if PA_SavedVars.Repair[activeProfile].enabled == true then
start index: 123, end index: 180
----
notation: -- early check if there is something to repair
line code: if GetRepairAllCost() > 0 then
start index: 233, end index: 262
----
notation: -- check if equipped items shall be repaired
line code: if PA_SavedVars.Repair[activeProfile].equipped then
start index: 315, end index: 365
----
notation: -- check if backpack items shall be repaired
line code: if PA_SavedVars.Repair[activeProfile].backpack then
start index: 509, end index: 559
----


In [23]:
from utils import search_files_for_substring
directory = '/home/uework/AiWeb/pwz/github/data/lua/shengqu'
substring = """g_mir.common.activityMgr"""

matches = search_files_for_substring(directory, substring)
for filename, lines in matches:
    print(f"Found in {filename} on lines: {', '.join(map(str, lines))}")

Found in /home/uework/AiWeb/pwz/github/data/lua/shengqu/logic/challenge_dragon/challenge_dragon_dup.lua on lines: 9
Found in /home/uework/AiWeb/pwz/github/data/lua/shengqu/logic/manager/activity_mgr.lua on lines: 321
Found in /home/uework/AiWeb/pwz/github/data/lua/shengqu/logic/personal_boss/personal_boss_dup.lua on lines: 11
Found in /home/uework/AiWeb/pwz/github/data/lua/shengqu/logic/shabak/shabak_dup.lua on lines: 12
Found in /home/uework/AiWeb/pwz/github/data/lua/shengqu/logic/swordSoul/swordSoulDup.lua on lines: 6, 8


In [1]:
from utils import *
# Get all Lua files in the directory
directory_path = '/home/uework/AiWeb/pwz/github/data/lua/shengqu'  # Replace with your directory path
results = find_all_samples(directory_path,mute=True)
# print(json.dumps(results, indent=4))
samples_to_lua_file(results,'output.lua')


/home/uework/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 857/857 [00:01<00:00, 559.22it/s] 
